In [1]:
import sys
sys.path.insert(0, '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages')
sys.path

['/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python36.zip',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/lib-dynload',
 '',
 '/home/mnetlab/.local/lib/python3.6/site-packages',
 '/home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages',
 '/home/mnetlab/.local/lib/python3.6/site-packages/IPython/extensions',
 '/home/mnetlab/.ipython']

In [2]:
# Modified fedavg for our input
from __future__ import absolute_import, division, print_function, unicode_literals

# Other import
import sys
import os
import time
import tensorflow as tf
from data.preprocessing import preprocessing_for_training, separate_and_preprocess_for_simple_fed, evaluate_with_new_model
from data.read_data import read_data, read_setting
from data.data_utils import load_cifar10_data, train_test_label_to_categorical
from model.model import init_model, record_history, training_once, print_result_for_fed
from model.operation import broadcast_to_device, caculate_delta, aggregate_add, aggregate_division_return 
from config.environment import gpu_decision, zmq_bind

from math import floor
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

import multiprocessing as mp
import zmq

Using TensorFlow backend.


In [3]:
def step_decay(epoch):
    epoch = _ + epo
    # initial_lrate = 1.0 # no longer needed
    drop = 0.99
    epochs_drop = 1.0
    
    lrate = 0.2 * pow(drop, floor((1+epoch)/epochs_drop))
    return lrate

In [4]:
_, epo = 0, 0

In [5]:
def gpu_decision(num_gpu ,device_num):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(device_num%num_gpu)
    with K.tf.device('/device:GPU:0'):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        sess = tf.Session(config=config)

In [6]:
def model_training(r, device_num):
    # Load CIFAR-10 data
    train_images, train_labels, test_images, test_labels = load_cifar10_data()
    
    gpu_decision(training_info["num_gpu"] ,device_num)
            
    model_m = tf.keras.models.load_model('parallel/model_m.h5')

    # Define our model
    if(r == 0):
        locals()['model_{}'.format(device_num)] = init_model()
    else:
        locals()['model_{}'.format(device_num)] = tf.keras.models.load_model('parallel/model_m.h5')
    
    # Local training on each device
    for epo in range(training_info["num_local_epoch"]):
        train_new_image, train_new_label = separate_and_preprocess_for_simple_fed(device_num, train_images, train_labels, training_info["num_device"])
        history_temp = training_once(locals()['model_{}'.format(device_num)], train_new_image, train_new_label, training_info, augment, callback)

    # Calculate delta weight on each device
    caculate_delta(locals()['model_{}'.format(device_num)], model_m)
        
    # Save delta weight in each model weight(.h5) file 
    try:
        if(r == 0):
            locals()['model_{}'.format(device_num)].save('parallel/model_{}.h5'.format(device_num))
        else:
            locals()['model_{}'.format(device_num)].save_weights('parallel/model_{}.h5'.format(device_num))
    except:
        print("error found")

In [ ]:
# Adjust parameters of the model
callback = tf.keras.callbacks.LearningRateScheduler(step_decay)
# Define the method for preprocessing
augment = ImageDataGenerator(preprocessing_function=preprocessing_for_training)

# bind zmq socket port
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5566")

 # Read detailed settings from json file
detailed_setting = read_setting()
training_info = detailed_setting["training_info"]

while True:
    while True:
        # Recieve message from server
        message = socket.recv()
        ms = str(message, encoding = "utf-8").split(":")
        r = int(ms[0])
        device_num = int(ms[1])
        print("(Received) round: %s, device: %s" % (message, device_num))
        
        # Start process for the device
        locals()['p_{}'.format(device_num)] = mp.Process(target=model_training, args=(r, device_num))
        locals()['p_{}'.format(device_num)].start()
        
        socket.send(b"ok")
        if(device_num == (training_info['num_device']-1)): break
    
    # Wait all processes done the jobs
    for device in range(training_info['num_device']):
        locals()['p_{}'.format(device_num)].join()

    # Tell server that the 'r' round finish
    message = socket.recv()
    socket.send(b"finish")
    if(r == (training_info['num_round']-1)): break

(Received) round: b'0:0', device: 0
(Received) round: b'0:1', device: 1
(Received) round: b'0:2', device: 2
(Received) round: b'0:3', device: 3
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructorWARNING:tensorflow:From /home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.WARNING:tensorflow:From /home/mnetlab/anaconda3

/home/mnetlab/Jeff/model/operation.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  model_m.layers[layer].get_weights()))
/home/mnetlab/Jeff/model/operation.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  model_m.layers[layer].get_weights()))
/home/mnetlab/Jeff/model/operation.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

(Received) round: b'1:0', device: 0
(Received) round: b'1:1', device: 1
(Received) round: b'1:2', device: 2
(Received) round: b'1:3', device: 3
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructorWARNING:tensorflow:From /home/mnetlab/anaconda3/envs/pp_fl/lib/python3.6/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

I